# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, sys
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break
    
from data_wrangling import specific_utils, process_monkey_information, general_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from neural_data_analysis.neural_analysis_tools.get_neural_data import neural_data_processing
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data, plot_modeling_result
from neural_data_analysis.neural_analysis_tools.model_neural_data import transform_vars, neural_data_modeling, drop_high_corr_vars, drop_high_vif_vars
from neural_data_analysis.topic_based_neural_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class, pn_aligned_by_seg, pn_aligned_by_event
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural.pn_decoding import pn_decoding_utils, plot_pn_decoding, pn_decoding_model_specs
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural.pn_decoding.interactions import add_interactions, discrete_decoders
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural.pn_decoding.interactions.band_conditioned import conditional_decoding_clf, conditional_decoding_reg, cond_decoding_plots
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class, cca_utils, cca_cv_utils
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_plotting import cca_plotting, cca_plot_lag_vs_no_lag, cca_plot_cv
from machine_learning.ml_methods import regression_utils, regz_regression_utils, ml_methods_class, classification_utils, ml_plotting_utils, ml_methods_utils
from planning_analysis.show_planning import nxt_ff_utils, show_planning_utils
from neural_data_analysis.neural_analysis_tools.gpfa_methods import elephant_utils, fit_gpfa_utils, plot_gpfa_utils, gpfa_helper_class
from neural_data_analysis.neural_analysis_tools.align_trials import time_resolved_regression, time_resolved_gpfa_regression,plot_time_resolved_regression
from neural_data_analysis.neural_analysis_tools.align_trials import align_trial_utils

import sys
import math
import gc
import subprocess
from pathlib import Path
from importlib import reload

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
import torch
from numpy import pi
from catboost import CatBoostRegressor

from sklearn.linear_model import RidgeCV
from sklearn.model_selection import cross_val_score

# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr

# Neuroscience specific imports
import neo
import rcca
import quantities as pq

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
print("done")

%load_ext autoreload
%autoreload 2

# retrieve data

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0416"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0321"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0329"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0403"

In [ ]:
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0312"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330"
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0316"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0327"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0328"

In [ ]:
reduce_y_var_lags = False
planning_data_by_point_exists_ok = True
y_data_exists_ok = True
bin_width = 0.1

pn = pn_aligned_by_event.PlanningAndNeuralEventAligned(raw_data_folder_path=raw_data_folder_path, bin_width=bin_width)
pn.prep_data_to_analyze_planning(planning_data_by_point_exists_ok=planning_data_by_point_exists_ok)
pn.planning_data_by_point, cols_to_drop = general_utils.drop_columns_with_many_nans(
    pn.planning_data_by_point)
pn.get_x_and_y_data_for_modeling(exists_ok=y_data_exists_ok, reduce_y_var_lags=reduce_y_var_lags)

# Define key features

## Get data (try only a few features right now)

In [ ]:
key_features = [
    'new_bin', 'new_segment', 'whether_test',
    'cur_eye_hor_l', 'cur_eye_ver_l', 'cur_eye_hor_r', 'cur_eye_ver_r',
    'nxt_eye_hor_l', 'nxt_eye_ver_l', 'nxt_eye_hor_r', 'nxt_eye_ver_r',
    'LDz', 'RDz', 'LDx', 'RDx',
    'gaze_mky_view_x', 'gaze_mky_view_y', 'gaze_mky_view_angle',
    'cur_opt_arc_dheading', 'cur_ff_distance',
    'cur_ff_rel_x', 'cur_ff_rel_y', 'nxt_ff_rel_x', 'nxt_ff_rel_y', 'nxt_ff_distance',
    'num_ff_visible', 'num_ff_in_memory',
    'cur_ff_distance_at_ref', 'cur_ff_angle_boundary_at_ref', 'nxt_ff_distance_at_ref',
    'speed', 'ang_speed', 'accel', 'ang_accel',
    'monkey_speeddummy', 'curv_of_traj',
    'angle_from_cur_ff_to_nxt_ff',
    'time_since_last_capture', 'bin_mid_time_rel_to_event',
    'time', 'target_index',
    'cur_vis', 'nxt_vis', 'nxt_in_memory', 'log1p_num_ff_visible',
]


In [ ]:
has_duplicates = len(key_features) != len(set(key_features)) 
print(has_duplicates)

dupes = {x for x in key_features if key_features.count(x) > 1}
print(dupes)

In [ ]:
pn.prepare_seg_aligned_data(start_t_rel_event=-0.25, end_t_rel_event=1.25, end_at_stop_time=False)

In [ ]:
pn.get_concat_data_for_regression(use_raw_spike_data_instead=True,
                                    apply_pca_on_raw_spike_data=False,
                                    use_lagged_raw_spike_data=False) 

In [ ]:
import numpy as np
import pandas as pd

pn.concat_behav_trials, added_cols = pn_decoding_utils.prep_behav(pn.concat_behav_trials)
pn.rebinned_behav_data, _ = pn_decoding_utils.prep_behav(pn.rebinned_behav_data)

key_features = list(set(key_features + added_cols))
key_features = [f for f in key_features if f in pn.concat_behav_trials.columns]

key_features2 = (['cur_ff_distance', 'log1p_cur_ff_distance', 'speed', 'accel', 'time_since_last_capture'] + added_cols)


In [ ]:
pn.concat_behav_trials = pn.concat_behav_trials[key_features].copy()
pn.rebinned_behav_data = pn.rebinned_behav_data[key_features].copy()
    

mask = pn.concat_behav_trials['bin_mid_time_rel_to_event'] > 0
pn.concat_behav_trials = pn.concat_behav_trials[mask]
pn.concat_neural_trials = pn.concat_neural_trials[mask]

print('concat_behav_trials:', pn.concat_behav_trials.shape)
print('rebinned_behav_data:', pn.rebinned_behav_data.shape)

print("Speed quantiles:")
for q in [0, 0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 1]:
    val = pn.concat_behav_trials['speed'].quantile(q)
    print(f"  {q:.2f}: {val:.5f}")



# Build interaction labels

## find ranges of vars

In [ ]:
temp_df = pn.planning_data_by_point.copy()
temp_df, _ = pn_decoding_utils.prep_behav(temp_df)

for var in ['speed', 'ang_speed', 'accel', 'ang_accel', 'cur_ff_distance', 'nxt_ff_distance', 'cur_ff_angle', 'nxt_ff_angle', 'cur_ff_rel_x', 'cur_ff_rel_y', 'cur_ff_distance_at_ref', 'log1p_cur_ff_distance', 'log1p_nxt_ff_distance']:
    print(f"Quantiles for {var}:")
    for q in [0, 0.01, 0.05, 0.1, 0.25, 0.5, 0.75, 0.9, 0.95, 0.99, 1]:
        val = temp_df[var].quantile(q)
        print(f"  {q:.2f}: {val:.5f}")
    print("\n")

## add bands

In [ ]:
flags = {
    'use_raw_spike_data_instead': True,
    'apply_pca_on_raw_spike_data': False,
    'use_lagged_raw_spike_data': False,
}

pn.get_concat_data_for_regression(**flags) 
df = pn.concat_behav_trials.copy()
df, added_cols = pn_decoding_utils.prep_behav(df)
df = add_interactions.add_behavior_bands(df)

## iterate

### continuous

In [ ]:
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural.pn_decoding.interactions.band_conditioned.specified_pairs import CONTINUOUS_INTERACTIONS, DISCRETE_INTERACTIONS


In [ ]:
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural.pn_decoding.interactions.band_conditioned import conditional_decoding_reg, conditional_decoding_clf, cond_decoding_plots


In [ ]:
reload(conditional_decoding_reg)
reload(cond_decoding_plots)

In [ ]:
#save_path = os.path.join(pn.planning_and_neural_folder_path, 'pn_decoding', 'band_conditioned_reg')
save_path = pn_decoding_utils.get_band_conditioned_save_path(pn, 'reg')
outs_reg = conditional_decoding_reg.run_band_conditioned_reg_decoding(df,
    pn.concat_neural_trials,
    CONTINUOUS_INTERACTIONS,
    max_pairs=100,
    save_path=save_path)

### discrete

In [ ]:
#save_path = os.path.join(pn.planning_and_neural_folder_path, 'pn_decoding', 'band_conditioned_clf')

save_path = pn_decoding_utils.get_band_conditioned_save_path(pn, 'clf')
outs_clf =conditional_decoding_clf.run_band_conditioned_clf_decoding(df,
    pn.concat_neural_trials,
    DISCRETE_INTERACTIONS,
    max_pairs=100,
    save_path=save_path)

## decode

In [ ]:
df = add_interactions.add_pairwise_interaction(
    df=df,
    var_a='speed_band',
    var_b='cur_ff_angle_band',
    new_col='speed_angle_state',
)

y_var, x_var = add_interactions.prune_rare_states_two_dfs(
    df,
    pn.concat_neural_trials,
    label_col='speed_angle_state',
    min_count=200
)

results_df = discrete_decoders.sweep_decoders_xy(
    x_df=x_var,
    y_df=y_var,
    label_col='speed_angle_state',
    model_types=['logreg', 'svm', 'ridge'],
)

summary_df = (
    results_df
    .groupby('model', as_index=False)
    .agg(mean_bal_acc=('balanced_accuracy', 'mean'))
)

## conditioned

In [ ]:
reload(discrete_decoders)

In [ ]:
results_df = conditional_decoding_clf.compare_component_conditioned_vs_global(
    x_df=x_var,
    y_df=y_var,
    target_col='cur_ff_angle_band',
    condition_col='speed_band',
    model_type='logreg',
)

summary_df = (
    results_df
    .groupby('context', as_index=False)
    .agg(mean_bal_acc=('balanced_accuracy', 'mean'))
)


In [ ]:
summary_df

## hier

In [ ]:
hier_df = discrete_decoders.hierarchical_decode_speed_angle(
    x_df=x_var,
    y_df=y_var,
    speed_col='speed_band',
    angle_col='cur_ff_angle_band',
    model_type='logreg',
)

hier_df

## cross_condition_decode

In [ ]:
rows = []

rows.append(discrete_decoders.cross_condition_decode(
    x_var, y_var,
    target_col='cur_ff_angle_band',
    condition_col='speed_band',
    train_conditions=['FAST', 'SLOW'],
    test_conditions=['CRUISE'],
))

rows.append(discrete_decoders.cross_condition_decode(
    x_var, y_var,
    target_col='cur_ff_angle_band',
    condition_col='speed_band',
    train_conditions=['CRUISE'],
    test_conditions=['FAST'],
))

cross_df = pd.DataFrame([r for r in rows if r is not None])
cross_df

## hyperparam tuning

In [ ]:

results_df = discrete_decoders.decode_with_param_sweep_xy(
    x_df=x_var,                         # neural features
    y_df=y_var,                         # behavior labels
    label_col='speed_angle_state',     # what you decode
    model_type='logreg',               # ONE model family
    n_splits=5,
)


## chance level

In [ ]:
def add_chance_level(results_df, df, label_col):
    probs = df[label_col].value_counts(normalize=True)
    chance = np.sum(probs ** 2)

    results_df = results_df.copy()
    results_df['chance'] = chance
    results_df['above_chance'] = results_df['accuracy'] - chance

    return results_df


In [ ]:
summary_df = (
    results_df
    .groupby(['label', 'model'], as_index=False)
    .agg(
        mean_acc=('accuracy', 'mean'),
        sem_acc=('accuracy', lambda x: x.std() / np.sqrt(len(x)))
    )
)


In [ ]:
len(pn.concat_behav_trials.columns)

In [ ]:
pn.planning_data_by_bin.columns.to_list()

In [ ]:
df = pn.concat_behav_trials.copy()

In [ ]:
df

In [ ]:
stop!